# [실습 5] Agentic RAG

학습 내용:
Agentic RAG는 기존 RAG를 기반으로, 동적으로 작업을 실행할 수 있는 자율적인 의사 결정 주체인 Agent를 도입한 것으로  
쿼리에 따라 필요한 도구를 불러서 사용할 수 있습니다.  
지금까지 만든 도구들을 가지고 하나의 Agent로 구축해보겠습니다.  


## 1. 환경 설정 및 기본 세팅
필요한 라이브러리들을 설치하고 import 합니다.  

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import tool
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

OpenAI API 키를 환경변수로 설정하여 GPT 모델을 사용할 수 있도록 준비합니다.

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI_API_KEY: ")
os.environ["TAVILY_API_KEY"] = getpass("Enter your TAVILY_API_KEY: ")

## 2. RAG 파이프라인 구축 (문서 로드, 분할, 임베딩, 벡터 스토어 생성)
Agent가 특정 전문 분야에 대해 깊이 있는 답변을 하려면, 해당 분야의 지식을 검색할 수 있는 내부 데이터베이스가 필요합니다.   
이 단계는 "Attention Is All You Need" 논문 PDF를 Agent가 참고할 수 있는 '지식 창고(Vector Store)'로 만드는 과정입니다.

1.  **데이터 로드 (Loading):**
    *   `PyPDFLoader`를 사용하여 웹에 있는 논문 PDF 파일을 불러옵니다. 이 단계는 Agent에게 학습시킬 원본 지식 데이터를 가져오는 첫걸음입니다.

2.  **텍스트 분할 (Splitting):**
    *   로드한 문서는 양이 방대하므로, LLM이 한 번에 처리할 수 있는 의미 있는 작은 단위, 즉 '청크(Chunk)'로 나눕니다.
    *   `RecursiveCharacterTextSplitter`는 문단의 의미가 최대한 유지되도록 문단, 문장 순으로 자릅니다. `chunk_overlap` 옵션은 청크 간의 내용이 일부 겹치게 하여, 분할 지점에서 문맥이 끊기는 것을 방지합니다.

3.  **임베딩 (Embedding):**
    *   분할된 텍스트 청크들을 컴퓨터가 이해할 수 있는 숫자 벡터(Vector)로 변환합니다.
    *   `HuggingFaceEmbeddings` (specifically `all-MiniLM-L6-v2` 모델)를 사용하여 각 청크의 '의미'를 벡터 공간의 특정 좌표로 맵핑합니다. **의미가 비슷한 텍스트는 벡터 공간에서도 가까운 위치에 존재하게 됩니다.**

4.  **벡터 스토어 구축 및 검색기 생성 (Storing & Retrieving):**
    *   변환된 모든 벡터들을 `FAISS`라는 고속 벡터 데이터베이스에 저장하고 인덱싱합니다. 이제 이 벡터 스토어는 우리의 '지식 창고' 역할을 합니다.
    *   마지막으로, 이 벡터 스토어에서 정보를 쉽게 꺼내 쓸 수 있도록 `as_retriever()`를 통해 '검색기(Retriever)'를 생성합니다. 이 검색기는 질문이 들어오면 질문의 의미 벡터와 가장 가까운 벡터(즉, 가장 관련성 높은 문서 청크)를 신속하게 찾아주는 역할을 합니다.

> **요약:** 원본 PDF 문서를 잘게 나누고, 각 조각의 의미를 숫자로 변환하여 검색이 용이한 데이터베이스에 저장함으로써, Agent가 특정 주제에 대해 빠르게 참조할 수 있는 **개인화된 검색 엔진**을 만든 것입니다.


In [ ]:
# 1. 데이터 로드 (PDF)
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"
loader = PyPDFLoader(pdf_url)
docs = loader.load()
print(f"- '{docs[0].metadata['source']}' 에서 {len(docs)}개 페이지 로드 완료")

# 2. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)
print(f"- {len(docs)}개 페이지를 {len(split_docs)}개의 청크로 분할 완료")

# 3. 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("- 임베딩 모델 'all-MiniLM-L6-v2' 로드 완료")

# 4. 벡터 스토어 생성 및 검색기(Retriever) 설정
vectorstore = FAISS.from_documents(split_docs, embedding_model)
retriever = vectorstore.as_retriever()
print("- FAISS 벡터 스토어 및 검색기 생성 완료")
print("RAG 파이프라인 구축 완료")
print("="*50)

## 3. 다중 도구(Tools) 생성
Agent는 스스로 웹 검색을 하거나 번역을 할 수 없습니다.   
이러한 기능들은 '도구(Tool)'의 형태로 만들어서 Agent에게 제공해야 합니다.   
이 단계에서는 Agent가 사용할 수 있는 4가지 핵심 능력을 정의하고 구현합니다.

**핵심 원리:**
각 도구에는 **이름(`name`)** 과 **설명(`description`)** 이 있습니다.   
Agent의 두뇌 역할을 하는 LLM은 사용자의 질문을 보고, 각 도구의 '설명'을 읽은 뒤, 어떤 도구를 사용하는 것이 가장 적절할지 판단합니다.    
**따라서 이 설명을 명확하고 상세하게 작성하는 것이 매우 중요합니다.**

*   **[도구 1] RAG 검색 도구 (`retriever_tool`):**
    *   위 `2단계`에서 만든 'Attention Is All You Need' 논문 전용 검색 엔진을 사용합니다.
    *   `create_retriever_tool` 함수를 통해 검색기(`retriever`)를 Agent가 사용할 수 있는 표준 도구 형태로 감쌉니다.
    *   Agent는 이 도구의 설명을 보고, 논문과 관련된 질문이 들어왔을 때 이 도구를 호출합니다.

*   **[도구 2] 웹 검색 도구 (`tavily_tool`):**
    *   실시간 인터넷 정보를 검색합니다.
    *   `TavilySearch`를 초기화하여 웹 검색 기능을 도구로 만듭니다.
    *   Agent는 최신 정보, 일반 상식 등 논문에 없는 내용을 질문받았을 때 이 도구를 선택합니다.

*   **[도구 3] 웹페이지 분석/요약 도구 (`process_and_summarize_webpage`):**
    *  사용자가 제공한 URL의 내용을 실시간으로 가져와 요약합니다.
    *  `@tool` 데코레이터를 사용하여 직접 만든 Python 함수를 Agent용 도구로 변환합니다.   
    * 이 함수는 URL을 입력받아 `WebBaseLoader`로 페이지 내용을 읽고, `load_summarize_chain`을 통해 LLM으로 내용을 요약하여 반환합니다.

*   **[도구 4] 번역 도구 (`translate_text`):**
    *  텍스트를 지정된 언어로 번역합니다.
    *  이 역시 `@tool` 데코레이터를 사용한 커스텀 함수입니다. 번역할 텍스트와 목표 언어를 입력받아, 번역 프롬프트와 LLM을 결합하여 번역 결과를 생성합니다.

In [ ]:
# 도구 내부에서 사용할 LLM (요약, 번역 등)
llm_for_tools = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- 도구 1: RAG 검색 도구 ---
retriever_tool = create_retriever_tool(
    retriever,
    "attention_is_all_you_need_search",
    "'Attention Is All You Need' 논문에 대한 정보를 검색하고 반환합니다. 논문의 저자, 초록, 방법론(self-attention, multi-head attention, transformer 등)과 관련된 모든 질문에 이 도구를 사용하세요."
)

# --- 도구 2: 웹 검색 도구 ---
tavily_tool = TavilySearchResults(
    max_results=3,
    description="최신 정보나 일반적인 사실에 대한 질문에 답하기 위해 웹을 검색합니다. 벡터 스토어에 없는 내용에 사용됩니다."
)

# --- 도구 3: 웹페이지 분석/요약 도구 ---
@tool
def process_and_summarize_webpage(url: str):
    """
    사용자가 제공한 URL의 웹페이지 콘텐츠를 로드하여 분석하고 그 내용을 요약합니다.
    URL과 관련된 질문에 답하거나 내용을 요약해야 할 때 사용해야 합니다.
    """
    try:
        loader = WebBaseLoader([url])
        docs = loader.load()
        summarize_chain = load_summarize_chain(llm=llm_for_tools, chain_type="stuff")
        summary = # [Your code]
        return summary
    except Exception as e:
        return f"오류가 발생했습니다: {e}"

# --- 도구 4: 번역 도구 ---
@tool
def translate_text(text: str, target_language: str):
    """
    주어진 텍스트를 지정된 대상 언어로 번역합니다.
    사용자가 번역을 요청할 때 사용해야 합니다.
    """
    prompt = ChatPromptTemplate.from_template("Translate the following text to {target_language}:\n\n{text}")
    translation_chain = # [Your code]
    return translation_chain. # [Your code]

print("="*50)

## 4. Agent 생성 및 연동
이제 준비된 '지식 창고'와 '도구들'을 사용하여 실제 작업을 수행할 Agent를 조립하고 실행 가능한 형태로 만드는 마지막 단계입니다.

1.  **도구 목록 통합 (Tool Integration):**
    *   앞서 만든 4개의 도구(`retriever_tool`, `tavily_tool` 등)를 하나의 Python 리스트(`tools`)에 담습니다. 이 리스트가 Agent에게 전달될 '공구함'입니다.

2.  **Agent의 두뇌(LLM) 선택 (LLM for Reasoning):**
    *   `ChatOpenAI` 모델을 `agent_llm`으로 초기화합니다. 이 LLM은 사용자의 질문에 직접 답하는 것이 아니라, **사용자의 질문 의도를 파악하고, 어떤 도구를 사용할지 '생각'하고 '결정'하는 추론(Reasoning)의 역할**을 담당합니다.

3.  **Agent 프롬프트 설정 (Prompting):**
    *   `hub.pull("hwchase17/openai-functions-agent")`를 통해 검증된 Agent용 프롬프트를 가져옵니다. 이 프롬프트는 Agent의 두뇌(LLM)에게 "너는 이제부터 보조원(Assistant)이야. 주어진 질문을 보고, 내가 준 도구 목록 중에서 가장 적합한 것을 골라 사용해. 그리고 그 결과를 바탕으로 최종 답변을 만들어." 와 같은 행동 지침을 제공합니다.

4.  **Agent 생성 (Agent Creation):**
    *   `create_openai_functions_agent` 함수에 **`두뇌(agent_llm)`**, **`공구함(tools)`**, **`행동 지침(prompt)`** 을 모두 전달하여 `agent`를 생성합니다. 이로써 논리적인 'Agent'가 완성됩니다.

5.  **Agent 실행기 생성 (Executor Creation):**
    *   마지막으로, 생성된 `agent`를 `AgentExecutor`로 감싸줍니다. `AgentExecutor`는 실제 실행을 담당하는 컨트롤러입니다.
    *   **작동 방식:**
        1.  사용자 질문을 `agent`에게 전달
        2.  `agent`(LLM)가 생각 후 사용할 도구와 입력값 결정 (예: "tavily_tool을 '최신 프론트엔드 프레임워크'라는 검색어로 호출해라")
        3.  `Executor`가 실제로 해당 도구를 실행하고 결과(검색 결과)를 얻음
        4.  결과를 다시 `agent`에게 전달
        5.  `agent`가 결과를 보고 최종 답변을 생성하거나, 필요시 다른 도구를 추가로 호출
        6.  이 과정이 반복되다 최종 답변이 완성되면 사용자에게 반환됩니다. `verbose=True`는 이 모든 중간 과정을 눈으로 볼 수 있게 해줍니다.

In [ ]:
# 1. 사용할 도구 목록 통합
tools = [
    # [Your code]
]

# 2. Agent가 추론에 사용할 LLM
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. Agent 프롬프트 가져오기
prompt = hub.pull("hwchase17/openai-functions-agent")

# 4. Agent 생성
agent = create_openai_functions_agent(agent_llm, tools, prompt)

# 5. Agent 실행기(Executor) 생성
agent_executor = AgentExecutor(
    # [Your code]
    verbose=True)

print("Agent 생성 및 연동 완료")

## 5. 통합 테스트

In [ ]:
# --- 예시 1: RAG 검색 ---
print("\n--- [Test 1] 논문 관련 질문 ---")
question1 = "'Attention Is All You Need' 논문에서 제안하는 트랜스포머 모델의 주요 구성 요소는 무엇인가요?"
response1 = agent_executor.invoke({"input": question1})
print("\n[최종 답변]:", response1["output"])

# --- 예시 2: 웹 검색 ---
print("\n--- [Test 2] 일반 상식 질문 ---")
question2 = "요즘 가장 인기 있는 프론트엔드 프레임워크는 무엇인가요?"
response2 = agent_executor.invoke({"input": question2})
print("\n[최종 답변]:", response2["output"])

# --- 예시 3: 웹페이지 요약 ---
print("\n--- [Test 3] 웹페이지 요약 질문 ---")
question3 = "https://www.zdnet.co.kr/view/?no=20240117100149 이 뉴스 기사의 내용을 세 문장으로 요약해줘."
response3 = agent_executor.invoke({"input": question3})
print("\n[최종 답변]:", response3["output"])

# --- 예시 4: 번역 ---
print("\n--- [Test 4] 번역 질문 ---")
question4 = "'Transformer 모델은 순환 신경망의 필요성을 완전히 제거했습니다.' 이 문장을 영어로 번역해줘."
response4 = agent_executor.invoke({"input": question4})
print("\n[최종 답변]:", response4["output"])

# --- 예시 5: 복합 질문 (RAG + 웹 검색) ---
print("\n--- [Test 5] 복합 질문 ---")
question5 = "'Attention Is All You Need' 논문의 저자 중 한 명인 Ashish Vaswani가 최근에 설립한 회사는 어디이고, 그 회사는 무엇을 하는 곳인가요?"
response5 = agent_executor.invoke({"input": question5})
print("\n[최종 답변]:", response5["output"])

### TO DO
웹 검색 도구를 테스트하는 [TEST 2]의 결과를 보면 2023년의 자료를 검색해온 것을 확인할 수 있습니다.  
도구를 생성할 때 작성된 'Description'을 보면 '최신 정보'라는 말이 기준이 없어서 모호했음을 유추해볼 수 있겠습니다.  
'Description' 내용을 더 구체적으로 작성하고, 다시 테스트 결과를 확인해보세요.   
아래는 'Description'을 수정한 후 결과 예시입니다.  

In [ ]:
# --- 예시 1: RAG 검색 ---
print("\n--- [Test 1] 논문 관련 질문 ---")
question1 = "'Attention Is All You Need' 논문에서 제안하는 트랜스포머 모델의 주요 구성 요소는 무엇인가요?"
response1 = agent_executor.invoke({"input": question1})
print("\n[최종 답변]:", response1["output"])

# --- 예시 2: 웹 검색 ---
print("\n--- [Test 2] 일반 상식 질문 ---")
question2 = "요즘 가장 인기 있는 프론트엔드 프레임워크는 무엇인가요?"
response2 = agent_executor.invoke({"input": question2})
print("\n[최종 답변]:", response2["output"])

# --- 예시 3: 웹페이지 요약 ---
print("\n--- [Test 3] 웹페이지 요약 질문 ---")
question3 = "https://www.zdnet.co.kr/view/?no=20240117100149 이 뉴스 기사의 내용을 세 문장으로 요약해줘."
response3 = agent_executor.invoke({"input": question3})
print("\n[최종 답변]:", response3["output"])

# --- 예시 4: 번역 ---
print("\n--- [Test 4] 번역 질문 ---")
question4 = "'Transformer 모델은 순환 신경망의 필요성을 완전히 제거했습니다.' 이 문장을 영어로 번역해줘."
response4 = agent_executor.invoke({"input": question4})
print("\n[최종 답변]:", response4["output"])

# --- 예시 5: 복합 질문 (RAG + 웹 검색) ---
print("\n--- [Test 5] 복합 질문 ---")
question5 = "'Attention Is All You Need' 논문의 저자 중 한 명인 Ashish Vaswani가 최근에 설립한 회사는 어디이고, 그 회사는 무엇을 하는 곳인가요?"
response5 = agent_executor.invoke({"input": question5})
print("\n[최종 답변]:", response5["output"])